In [1]:
import pandas as pd
import numpy as np
import importlib.metadata
import mysql.connector
import streamlit as st
import os

from datetime import datetime

In [2]:
# Cargar csv
df_combustibles = pd.read_csv("../bin/listado_combustibles.csv")
df_concesionarios = pd.read_csv("../bin/listado_concesionarios.csv")
df_distintivo_ambiental = pd.read_csv("../bin/listado_distintivo_ambiental.csv")
df_marcas = pd.read_csv("../bin/listado_marcas.csv")
df_modelos = pd.read_csv("../bin/listado_modelos.csv")
df_provincias = pd.read_csv("../bin/listado_provincias.csv")
df_sobrealimentacion = pd.read_csv("../bin/listado_sobrealimentacion.csv")
df_traccion = pd.read_csv("../bin/listado_traccion.csv")
df_cambio = pd.read_csv("../bin/listado_tipos_cambio.csv")
df_prestaciones = pd.read_csv("../bin/prestaciones.csv")

In [3]:
df_prestaciones['fecha_extraccion'] = pd.to_datetime(df_prestaciones['fecha_extraccion'], format='%d/%m/%Y', errors='coerce' )
df_prestaciones['fecha_extraccion'] = df_prestaciones['fecha_extraccion'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [4]:
#Comprobar si hay nans
df_prestaciones.isna().sum()

id_provincia                  1
fecha_extraccion              0
kilometraje                 116
id_concesionario              0
id_distintivo_ambiental     324
garantia                    626
precio_contado                0
precio_nuevo               1096
largo                      1143
ancho                      1133
alto                       1147
capacidad_maletero         1281
num_plazas                 1129
batalla                    1139
peso                        896
num_puertas                  44
consumo_medio              3247
consumo_carretera          3614
consumo_urbano             3614
deposito                   1571
cilindrada                 1814
id_traccion                1129
num_marchas                 935
potencia_kw                1119
potencia_cv                  46
par                        1245
velocidad_max              1268
aceleracion                1252
mes_matriculacion             0
ano_matriculacion             0
co2                        3617
num_cili

In [5]:
#Convertir a nan a none
for columna in df_prestaciones.columns:
    df_prestaciones[columna] = df_prestaciones[columna].replace([np.nan, pd.NA], None)

In [6]:
directorio_pkl = "../bin/imagenes_coches/"
os.makedirs(directorio_pkl, exist_ok=True)

archivos_pkl = sorted([f for f in os.listdir(directorio_pkl) if f.endswith(".pkl")])
dfs = [pd.read_pickle(os.path.join(directorio_pkl, archivo)) for archivo in archivos_pkl]

df_concatenado = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame(columns=["link_anuncio", "foto_binaria"])

df_concatenado = df_concatenado.reset_index(drop=True).reset_index()
df_concatenado = df_concatenado.rename(columns={"index": "id_coche"})
df_concatenado["id_coche"] += 1

In [7]:
# Conectar a la base de datos usando los valores almacenados en `secrets.toml`
def conectar_base_datos():
    conn = mysql.connector.connect(
        host=st.secrets["database"]["host"],
        user=st.secrets["database"]["user"],
        password=st.secrets["database"]["password"],
        database=st.secrets["database"]["database"]
    )
    return conn

In [11]:
contraseña_miguel = "password"
contraseña_moran = 'Deportivo96'
contraseña_carlos = "Whereisbotus1996"

In [16]:
def insertar_datos_en_lotes(df, nombre_tabla, lote=100):
    # Conexión a la base de datos MySQL
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password=contraseña_miguel,
        database="coches_segunda_mano"
    )
    cursor = conn.cursor()

    columnas = df.columns.tolist()
    query = f"""
        INSERT INTO {nombre_tabla} ({', '.join(columnas)})
        VALUES ({', '.join(['%s'] * len(columnas))})
    """

    # Dividir el DataFrame en lotes
    for i in range(0, len(df), lote):
        valores = [tuple(row) for row in df.iloc[i:i + lote].to_numpy()]
        cursor.executemany(query, valores)
        conn.commit()

    cursor.close()
    conn.close()
    print(f"Datos insertados en {nombre_tabla} correctamente.")

In [18]:
# Llamar a la funcion para insertar los datos en la tabla
insertar_datos_en_lotes(df_combustibles, 'combustibles')
insertar_datos_en_lotes(df_concesionarios, 'concesionarios')
insertar_datos_en_lotes(df_distintivo_ambiental, 'distintivos_ambientales')
insertar_datos_en_lotes(df_marcas, 'marcas')
insertar_datos_en_lotes(df_modelos, 'modelos')
insertar_datos_en_lotes(df_provincias, 'provincias')
insertar_datos_en_lotes(df_sobrealimentacion, 'sobrealimentaciones')
insertar_datos_en_lotes(df_traccion, 'tipo_traccion')
insertar_datos_en_lotes(df_cambio, 'tipos_cambio')
insertar_datos_en_lotes(df_concatenado, 'links_coches')
insertar_datos_en_lotes(df_prestaciones, 'prestaciones')

Datos insertados en combustibles correctamente.
Datos insertados en concesionarios correctamente.
Datos insertados en distintivos_ambientales correctamente.
Datos insertados en marcas correctamente.
Datos insertados en modelos correctamente.
Datos insertados en provincias correctamente.
Datos insertados en sobrealimentaciones correctamente.
Datos insertados en tipo_traccion correctamente.
Datos insertados en tipos_cambio correctamente.
Datos insertados en links_coches correctamente.
Datos insertados en prestaciones correctamente.


## Función para extraer datos

In [20]:
# Funcion para extraer datos
def extraer_datos(nombre_tabla, columna = *):
    
    try:
        # Conexión a la base de datos de MySQL
        conn = mysql.connector.connect(
            host = "localhost",
            user = "root",
            password = contraseña_moran,
            database = "coches_segunda_mano"
        )
        # Crear un cursor para ejecutar comandos SQL
        cursor = conn.cursor()

        # Crear la consulta SQL para seleccionar todos los datos de la tabla
        query = f"SELECT {columna} FROM {nombre_tabla}"

        # Ejecutar la consulta y recuperar los datos
        cursor.execute(query)
        resultado = cursor.fetchall()

        # Obtener los nombres de las columnas de la tabla
        columnas = [desc[0] for desc in cursor.description]

        # Convertir los resultados a un DataFrame
        df = pd.DataFrame(resultado, columns=columnas)

        # Cerrar el cursor y la conexión
        cursor.close()
        conn.close()

        # Devolver el DataFrame
        print(f"Datos de {nombre_tabla} extraidos correctamente")
        return df
    
    except mysql.connector.Error as e:
        print(f"Error al conectar con la base de datos: {e}")
        return pd.DataFrame() 
    
    finally:
        # Asegurarse de cerrar la conexión y el cursor
        if 'cursor' in locals():
            cursor.close()
        if 'conn' in locals():
            conn.close()

SyntaxError: invalid syntax (2200117266.py, line 2)

In [ ]:
df_prestaciones = extraer_datos("prestaciones")

In [ ]:
df_prestaciones['fecha_extraccion'] = pd.to_datetime(df_prestaciones['fecha_extraccion']).dt.strftime('%d/%m/%Y')

## Conexión streamlit

In [ ]:
# Función para conectar a la base de datos y comprobar la conexión con streamlit
def comprobar_conexion():
    try:
        # Intentar conectarse a la base de datos
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="Whereisbotus1996",
            database="coches_segunda_mano"
        )
        # Si la conexión es exitosa
        conn.close()
        return True
    except mysql.connector.Error as err:
        # Si ocurre un error
        return False

# Interfaz de Streamlit
st.title("Conexión a la Base de Datos")

# Botón para comprobar la conexión
if st.button("Comprobar conexión a la BBDD"):
    if comprobar_conexion():
        st.success("Conexión exitosa a la base de datos.")
    else:
        st.error("Error al conectar a la base de datos.")